In [3]:
pip install python_speech_features

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple/
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install hmmlearn

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple/
     |████████████████████████████████| 162 kB 23.8 MB/s eta 0:00:01
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [69]:
import os
import pickle
import numpy as np
import scipy.io.wavfile as wvf
from python_speech_features import mfcc
from hmmlearn.hmm import GMMHMM
import heapq

In [23]:
#训练数据路径
train_data_path =os.path.join(os.getcwd(),"datas/train/speech" )
label_path =os.path.join(os.getcwd(),"datas/labels/trainprompts_m" )
#测试数据路径
test_data_path =os.path.join(os.getcwd(),"datas/test/speech" )
#模型保存路径
model_path="hmm_gmm_model.pkl"

In [71]:
def wav2mfcc(labels,data_paths):
 
 trng_data = {} 
 for label,data_path in zip(labels,data_paths):
 # if label in trng_data.keys():
 # print(label)
    write_pickle = True 
    mfccs = []
    rate, sig = wvf.read(data_path)
    mfcc_feat = mfcc(sig, rate)
    mfccs.append(mfcc_feat)
    trng_data[label] = mfccs
 
 return trng_data

In [72]:
def obtain_config(labels):
 conf = {} 
 for label in labels:
  conf[label] = {}
  conf[label]["n_components"] = 2
  conf[label]["n_mix"] = 2
 return conf

In [73]:
def get_hmm_gmm(trng_datas=None, GMM_configs=None,model_path ="hmm_gmm_model.pkl",from_file=False):
 hmm_gmm = {}
 if not from_file:
  for label, trng_data in trng_datas.items():
   GMM_config = GMM_configs[label]
 
   hmm_gmm[label] = GMMHMM(n_components=GMM_config["n_components"],n_mix=GMM_config["n_mix"])
   if trng_data:
    hmm_gmm[label].fit(np.vstack(trng_data))
 
  pickle.dump(hmm_gmm, open(model_path, "wb"))
 else:
  hmm_gmm = pickle.load(open(model_path, "rb"))
 return hmm_gmm

In [75]:
def train(train_data_path,label_path,model_path):
 
 with open(os.path.join(label_path)) as f:
  labels = f.readlines() # 读取所有行，返回列表
 data_paths = [train_data_path+'/'+line.split()[0]+'.wav' for line in 
labels]
 labels = [ ' '.join(line.split()[1:]).strip() for line in labels]
 train_datas = wav2mfcc(labels,data_paths)
 GMM_configs = obtain_config(labels)
 hmm_gmm = get_hmm_gmm(train_datas, GMM_configs, 
model_path)
 return hmm_gmm
hmm_gmm = train(train_data_path,label_path,model_path)
print(hmm_gmm)

{'开 灯 七 小时': GMMHMM(algorithm='viterbi', covariance_type='diag',
       covars_prior=array([[[-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
         -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
         -1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
         -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
         -1.5, -1.5, -1.5]]]),
       covars...
       means_prior=array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
       means_weight=array([[0., 0.],
       [0., 0.]]), min_covar=0.001,
       n_components=2, n_iter=10, n_mix=2, params='stmcw', random_state=None,
       startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False,
       we

In [94]:
import numpy as np
import struct
import pickle
import heapq
from hmmlearn.hmm import GMMHMM  # 确保你安装了hmmlearn库

def read_mfc_file(file_path, num_features):
    with open(file_path, 'rb') as f:
        data = f.read()
        num_floats = len(data) // 4  # 每个浮点数占4个字节
        mfcc_feat = struct.unpack('f' * num_floats, data)
        mfcc_feat = np.array(mfcc_feat).reshape(-1, num_features)
    return mfcc_feat

def get_nbest(d, n):
    return heapq.nlargest(n, d, key=lambda k: d[k])

def test_mfcc(mfcc_feat, hmm_gmm):
    pred = {}
    for model in hmm_gmm: 
        pred[model] = hmm_gmm[model].score(mfcc_feat)
    return get_nbest(pred, 1), pred  # 获取得分最高的模型

def predict_label(mfcc_feat, hmm_gmm): 
    predicted, _ = test_mfcc(mfcc_feat, hmm_gmm)
    return predicted

# 加载训练好的模型
model_path = "hmm_gmm_model.pkl"
with open(model_path, 'rb') as f:
    hmm_gmm = pickle.load(f)

# 读取.mfc文件
# test1:关掉阀门六小时
mfcc_file_path = "datas/test/feature/ljkTest1.mfc"
num_features = 13  # 假设每帧有13个MFCC特征
mfcc_feat = read_mfc_file(mfcc_file_path, num_features)

# 使用MFCC特征进行预测
predicted = predict_label(mfcc_feat, hmm_gmm)
print("PREDICTED: %s" % predicted[0])

PREDICTED: 关掉 阀门 七 小时


In [87]:
import numpy as np
import struct
import pickle
import heapq
from hmmlearn.hmm import GMMHMM  # 确保你安装了hmmlearn库

def read_mfc_file(file_path, num_features):
    with open(file_path, 'rb') as f:
        data = f.read()
        num_floats = len(data) // 4  # 每个浮点数占4个字节
        mfcc_feat = struct.unpack('f' * num_floats, data)
        mfcc_feat = np.array(mfcc_feat).reshape(-1, num_features)
    return mfcc_feat

def get_nbest(d, n):
    return heapq.nlargest(n, d, key=lambda k: d[k])

def test_mfcc(mfcc_feat, hmm_gmm):
    pred = {}
    for model in hmm_gmm: 
        pred[model] = hmm_gmm[model].score(mfcc_feat)
    return get_nbest(pred, 1), pred  # 获取得分最高的模型

def predict_label(mfcc_feat, hmm_gmm): 
    predicted, _ = test_mfcc(mfcc_feat, hmm_gmm)
    return predicted

# 加载训练好的模型
model_path = "hmm_gmm_model.pkl"
with open(model_path, 'rb') as f:
    hmm_gmm = pickle.load(f)

# 读取.mfc文件
# test2:打开风扇八分钟
mfcc_file_path = "datas/test/feature/ljkTest2.mfc"
num_features = 13  # 假设每帧有13个MFCC特征
mfcc_feat = read_mfc_file(mfcc_file_path, num_features)

# 使用MFCC特征进行预测
predicted = predict_label(mfcc_feat, hmm_gmm)
print("PREDICTED: %s" % predicted[0])

PREDICTED: 打开 风扇 八 分钟
